In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
pd.set_option('display.max_columns', None)

In [101]:
df = pd.read_csv('data/dk_monthly_all_1768_2020.csv', delimiter=";")

In [136]:
# Only keep the Precipitation (has the value 601 in the column "elem_no")
df = df[df['elem_no'] == 601]

# Only one measuring station collected data before 1875. I will therefore only keep data from 1875 and onwards
df = df[df['year'] > 1874]  

# Change the decimal separators in the annual column from "," to "." – this is the column I am interested in right now
df['annual'] = df['annual'].replace(',','.',regex=True)

# Change the data type of the annual column from string to float
df = df.astype({'annual': 'float64'})

df_year = df[['year','annual']]

In [137]:
df_year.sort_values(by=['year'], inplace=True)

# Group by year and get the average rainfall for each year
df_year = df_year.groupby(['year']).mean('annual').reset_index().sort_values(by='year', ascending=True)

df_year['five_year' ] = df_year.annual.rolling( 5).mean()

df_year['ten_year' ] = df_year.annual.rolling( 10).mean()

df_year.to_csv('data/dk_yearly_averages_1974_2020.csv', index=False)
df_year

,year,annual,five_year,ten_year
0,1875,511.94,NaN,NaN
1,1876,553.02,NaN,NaN
2,1877,739.62,NaN,NaN
3,1878,620.52,NaN,NaN
4,1879,621.78,609.376,NaN
...,...,...,...,...
141,2016,590.92,711.976,705.524
142,2017,791.50,720.912,707.926
143,2018,568.88,711.788,694.652
144,2019,791.92,713.964,711.396


Explain the issues with data before the 50s

In [104]:
df_year50 = df_year[df_year['year'] > 1949]
df_year50.to_csv('data/dk_yearly_1950_2020.csv', index=False)

In [105]:
# Change the data type of the annual column from string to float

df_rain_monthly = pd.melt(df, id_vars=['year'], value_vars=['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'], ignore_index=False)

df_rain_monthly['value'] = df_rain_monthly['value'].replace(',','.',regex=True)
df_rain_monthly = df_rain_monthly.astype({'value': 'float64'})

df_rain_monthly
df_rain_monthly.to_csv('data/dk_monthly_1874_2020.csv', index=False)

In [125]:
df_rain_monthly50 = df_rain_monthly[df_rain_monthly['year'] > 1949]
df_rain_monthly50

df_rain_monthly50['decade'] = df_rain_monthly50['year'].apply(lambda x: '19' + str(x)[2:3] + '0')
df_rain_monthly50['decade'] = df_rain_monthly50['decade'].replace('1900','2000',regex=True)
df_rain_monthly50['decade'] = df_rain_monthly50['decade'].replace('1910','2010',regex=True)
df_rain_monthly50['decade'] = df_rain_monthly50['decade'].replace('1920','2020',regex=True)

df_rain_monthly50.rename(columns={"variable": "month", "value": "rainfall"}, inplace=True)

df_rain_monthly50['month_num'] = df_rain_monthly50['month'].apply(lambda x: '01' if x == 'jan' else '02' if x == 'feb' else '03' if x == 'mar' else '04' if x == 'apr' else '05' if x == 'may' else '06' if x == 'jun' else '07' if x == 'jul' else '08' if x == 'aug' else '09' if x == 'sep' else '10' if x == 'oct' else '11' if x == 'nov' else '12')

#df_rain_monthly50 = df_rain_monthly50.astype({'decade': 'int64'})
df_rain_monthly50 = df_rain_monthly50.astype({'month_num': 'int64'})

df_rain_monthly50.to_csv('data/dk_monthly_1950_2020.csv', index=False)

df_rain_monthly50.dtypes

year           int64
month         object
rainfall     float64
decade        object
month_num      int64
dtype: object

Looking at daily rain in Copenhagen

In [140]:
df = pd.read_csv('data/dk_daily_5735_601.csv', delimiter=";")

df

,stat_no,year,month,day,hour,elem_val
0,30370,1961,1,1,8,"0,7"
1,30370,1961,1,2,8,"2,5"
2,30370,1961,1,3,8,"0,1"
3,30370,1961,1,4,8,9
4,30370,1961,1,5,8,"0,1"
...,...,...,...,...,...,...
21910,5735,2020,12,27,23,"4,1"
21911,5735,2020,12,28,23,"1,1"
21912,5735,2020,12,29,23,"2,9"
21913,5735,2020,12,30,23,3
